In [1]:
#!pip install --upgrade oct2py
import numpy as np
import pandas as pd
from oct2py import octave
import matplotlib.pyplot as plt
import os
from scipy.optimize import root
import pickle
octave.addpath('C:/dynare/6.1/matlab') 
from IPython.display import display
#octave.addpath('/opt/homebrew/opt/dynare/lib/dynare/matlab')

    _pyeval at line 57 column 30

    _pyeval at line 57 column 30



`DyPy` is a class that we create and store in a file called `MyPy.py`. It contains methods for 

* converting data from a Dynare stucture to Numpy arrays.
* analyzing the data. 

See the file for details. 

In [2]:
from MyPy_2 import DyPy

Creates a file called `t_call.m`. This informs Dynare of the format we want for output. It is put here so that adjusments can be made easily; for example the number of periods can be changed here rather than by looking at the `.mod` file. It is included in the `.mod` file via: `@#include "t_call.m"` in `model_1.mod`

* `order`: the order of approximation for the stochastic simulation. order=1 is a fsrst-order approximation (linearization) Higher-order approximations (e.g., order=2 for second-order) capture nonlinear effects in the model, but they require more computational resources.
* `periods`: number of periods (time steps) for which the simulation will be run. periods=1000 means that the model will be simulated for 1000 periods. The default value is 0, which means no simulation will be performed.
* `nograph`: suppresses the graphical output of the simulation results in Dynare since we draw them separatetly. 
* `irf`: the number of periods for which the impulse response functions (IRFs) will be computed and plotted. 

In [3]:
with open('t_call.m', 'w') as file:
    file.write("stoch_simul(nograph, order=1, periods=10000, irf=100);")

 `make_model.ipyn` writes a file called `t_model.m`. `t_model.m`is the `model` block in the Dynare `.mod` file. It can instead be created directlty by commenting out the next `%run` line and coding the model. `make_model.ipyn` simplifies model creation when many of the lines are very similar to each other. Then they can be written in for loops. For the model we are currently working with, there is not much advantage to writing the file via `make_model` but I have done so nonetheless. 
 
 The created file is included in the `.mod` file via `@#include "t_model.m"` in `model_1.mod`. 

In [4]:
%run make_model.ipynb

`make_files.ipynb` writes the other Dynare blocks. See that file for details. However, note that in doing so, it solves for the steady state of the model using `root` from `scipy`. If the results below do not indicate success, the `.mod` file may not run correctly. Also, note that though we solve for the steady state here, we do not give Dynare the steady state but rather use these as starting values for their ss solver. I will later code a verification the steady states are the same. 

In [5]:
%run make_files_1.ipynb

eps,tht,alf,bet,om_0,om_1,om_2,delt,psi_n,psi_k,var_a_0,var_a_1,var_a_2,rho_a_0,rho_a_1,rho_a_2


Tells octave where to find the .m file it will run. The file is `call_dynare`. Set to wherever you have this file. 




In [6]:
#a=octave.addpath(os.path.expanduser('C:/Users/blankenw/GitHub/dynare_progs/np_1'))

Makes a call to Octave, which makes a call to Dynare. Then the `.mod` file is run in Dynare. Output from Dynare is displayed as it runs.

In [7]:

# A list of dictionaries representing updates to apply to my_dict:
new_pars = [{'psi_n': 0, 'psi_k': 0}, 
           {'psi_n': 2, 'psi_k': 0}, 
           {'psi_n': 0, 'psi_k': 2}]



# Create an empty dictionary to store your variables
t=[]
with open('mf1.pkl', 'wb') as f:
    pickle.dump((ss_params,model_params,ss_shock), f)


for i, update in enumerate(new_pars):
    model_params.update(update)
    with open('mf1.pkl', 'wb') as f:
        pickle.dump((ss_params,model_params,ss_shock), f)
    %run make_files_2.ipynb
    with open('mf2.pkl', 'rb') as f:
        ss_dict = pickle.load(f)
    # Set up Octave session and run your model
    octave.dynare('model_1.mod')

    # Pull necessary structures
    oo = octave.pull('oo_')
    Mthing = octave.pull('M_')
    instance = DyPy(oo,ss_dict)
    if i==0:
        oo_0=oo
    t.append(instance)


 message: The solution converged.
 success: True
  status: 1
     fun: [ 0.000e+00  0.000e+00 ...  2.634e-11 -4.198e-12]
       x: [ 2.663e-01  3.464e-01 ...  5.055e-01  1.215e-01]
    nfev: 61
    fjac: [[-1.159e-01  5.538e-11 ... -7.075e-04 -1.825e-03]
           [-1.610e-01 -1.959e-10 ...  1.344e-01 -4.294e-03]
           ...
           [-3.383e-01 -1.639e-02 ... -3.084e-01 -3.217e-01]
           [-6.421e-02  9.228e-02 ...  2.426e-02  2.031e-02]]
       r: [-8.630e+00  2.177e-01 ...  2.101e-01 -7.394e-01]
     qtf: [ 5.891e-09  1.761e-08 ... -7.917e-10  1.706e-10]
{'n_0': -1.3231119683891552, 'n_1': -1.0601505410798138, 'n_2': -0.9485787651498407, 's_0': -1.323111968359227, 's_1': -1.0601505409046252, 's_2': -0.9485787653271133, 'k': 0.8881260236195211, 'k_0': 0.8881260236466464, 'k_1': 0.8881260235509693, 'k_2': 0.8881260236279862, 'c_0': -1.4868599143386987, 'c_1': -0.7937127338831573, 'c_2': -0.6821409581254703, 'i': -2.1076062499344745, 'y_0': -1.0566741613194897, 'y': 0.2664378

C:\Users\blankenw\AppData\Local\Temp\ipykernel_3920\781936935.py:17: RuntimeWarning: invalid value encountered in scalar power
  fn_0=np.exp(a_0)*(1-alf)*(s_0/n_0/q_0)**alf*k**alf
C:\Users\blankenw\AppData\Local\Temp\ipykernel_3920\781936935.py:21: RuntimeWarning: invalid value encountered in scalar power
  fs_0=np.exp(a_0)*alf*(n_0/s_0*q_0)**(1-alf)*k**alf
C:\Users\blankenw\AppData\Local\Temp\ipykernel_3920\781936935.py:47: RuntimeWarning: invalid value encountered in scalar power
  eq12=c_0+i-np.exp(a_0)*n_0**(1-alf)*s_0**alf*k**alf


Starting Dynare (version 6.1).
Calling Dynare with arguments: none
Starting preprocessing of the model file ...
Found 32 equation(s).
Evaluating expressions...
Computing static model derivatives (order 1).
Normalizing the static model...
Finding the optimal block decomposition of the static model...
15 block(s) found:
  14 recursive block(s) and 1 simultaneous block(s).
  the largest simultaneous block has 11 equation(s)
                                 and 11 feedback variable(s).
Computing dynamic model derivatives (order 1).
Normalizing the dynamic model...
Finding the optimal block decomposition of the dynamic model...
13 block(s) found:
  12 recursive block(s) and 1 simultaneous block(s).
  the largest simultaneous block has 11 equation(s)
                                 and 11 feedback variable(s).
Preprocessing completed.
Preprocessing time: 0h00m01s.

STEADY-STATE RESULTS:

n_0   		 -1.70401
n_1   		 -0.984499
n_2   		 -0.810981
s_0   		 -0.622654
s_1   		 -1.34956
s_2   		 -1

C:\Users\blankenw\AppData\Local\Temp\ipykernel_3920\781936935.py:17: RuntimeWarning: invalid value encountered in scalar power
  fn_0=np.exp(a_0)*(1-alf)*(s_0/n_0/q_0)**alf*k**alf
C:\Users\blankenw\AppData\Local\Temp\ipykernel_3920\781936935.py:21: RuntimeWarning: invalid value encountered in scalar power
  fs_0=np.exp(a_0)*alf*(n_0/s_0*q_0)**(1-alf)*k**alf
C:\Users\blankenw\AppData\Local\Temp\ipykernel_3920\781936935.py:47: RuntimeWarning: invalid value encountered in scalar power
  eq12=c_0+i-np.exp(a_0)*n_0**(1-alf)*s_0**alf*k**alf


 message: The solution converged.
 success: True
  status: 1
     fun: [ 0.000e+00  0.000e+00 ...  2.634e-11 -4.198e-12]
       x: [ 2.663e-01  3.464e-01 ...  5.055e-01  1.215e-01]
    nfev: 61
    fjac: [[-1.159e-01  5.538e-11 ... -7.075e-04 -1.825e-03]
           [-1.610e-01 -1.959e-10 ...  1.344e-01 -4.294e-03]
           ...
           [-3.383e-01 -1.639e-02 ... -3.084e-01 -3.217e-01]
           [-6.421e-02  9.228e-02 ...  2.426e-02  2.031e-02]]
       r: [-8.630e+00  2.177e-01 ...  2.101e-01 -7.394e-01]
     qtf: [ 5.891e-09  1.761e-08 ... -7.917e-10  1.706e-10]
{'n_0': -1.3231119683891552, 'n_1': -1.0601505410798138, 'n_2': -0.9485787651498407, 's_0': -1.323111968359227, 's_1': -1.0601505409046252, 's_2': -0.9485787653271133, 'k': 0.8881260236195211, 'k_0': 0.8881260236466464, 'k_1': 0.8881260235509693, 'k_2': 0.8881260236279862, 'c_0': -1.4868599143386987, 'c_1': -0.7937127338831573, 'c_2': -0.6821409581254703, 'i': -2.1076062499344745, 'y_0': -1.0566741613194897, 'y': 0.2664378

C:\Users\blankenw\AppData\Local\Temp\ipykernel_3920\781936935.py:17: RuntimeWarning: invalid value encountered in scalar power
  fn_0=np.exp(a_0)*(1-alf)*(s_0/n_0/q_0)**alf*k**alf
C:\Users\blankenw\AppData\Local\Temp\ipykernel_3920\781936935.py:21: RuntimeWarning: invalid value encountered in scalar power
  fs_0=np.exp(a_0)*alf*(n_0/s_0*q_0)**(1-alf)*k**alf
C:\Users\blankenw\AppData\Local\Temp\ipykernel_3920\781936935.py:47: RuntimeWarning: invalid value encountered in scalar power
  eq12=c_0+i-np.exp(a_0)*n_0**(1-alf)*s_0**alf*k**alf


 message: The solution converged.
 success: True
  status: 1
     fun: [ 0.000e+00  0.000e+00 ...  2.634e-11 -4.198e-12]
       x: [ 2.663e-01  3.464e-01 ...  5.055e-01  1.215e-01]
    nfev: 61
    fjac: [[-1.159e-01  5.538e-11 ... -7.075e-04 -1.825e-03]
           [-1.610e-01 -1.959e-10 ...  1.344e-01 -4.294e-03]
           ...
           [-3.383e-01 -1.639e-02 ... -3.084e-01 -3.217e-01]
           [-6.421e-02  9.228e-02 ...  2.426e-02  2.031e-02]]
       r: [-8.630e+00  2.177e-01 ...  2.101e-01 -7.394e-01]
     qtf: [ 5.891e-09  1.761e-08 ... -7.917e-10  1.706e-10]
{'n_0': -1.3231119683891552, 'n_1': -1.0601505410798138, 'n_2': -0.9485787651498407, 's_0': -1.323111968359227, 's_1': -1.0601505409046252, 's_2': -0.9485787653271133, 'k': 0.8881260236195211, 'k_0': 0.8881260236466464, 'k_1': 0.8881260235509693, 'k_2': 0.8881260236279862, 'c_0': -1.4868599143386987, 'c_1': -0.7937127338831573, 'c_2': -0.6821409581254703, 'i': -2.1076062499344745, 'y_0': -1.0566741613194897, 'y': 0.2664378

Oct2PyError: Octave evaluation error:
error: Impossible to find the steady state (the sum of squared residuals of the static equations is 7.2361). Either the model doesn't have a steady state, there are an infinity of steady states, or the guess values are too far from the solution
error: called from:
    print_info at line 33, column 5
    steady at line 143, column 5
    driver at line 1068, column 1
    dynare at line 310, column 5

Dynare creates a `structure` which we will need so we `pull` it.

The output from Dynare is stored in a file called `oo_`. This brings `oo_` in as `oo`.

Draw all impulse response functions. 

In [ ]:
def plot_dta(instances, attributes,periods=100):
    plt.figure(figsize=(15, 5))
    for j, attribute in enumerate(attributes):
        # Create a subplot for this attribute
        plt.subplot(1, len(attributes), j+1)
        for i, instance in enumerate(instances):
            # Get the attribute value (time series data)
            attr_value = getattr(instance.ir_level, attribute)
            attr_value = attr_value.reshape(-1, 1)
            length = attr_value.shape[0]
            tm = np.linspace(1, length, length)
            # Add a line to the plot for this time series
            plt.plot(tm[0:periods], attr_value[0:periods], label=labels[i])

        plt.title(attribute)
        plt.legend()

    # Show the figure with all subplots


In [ ]:
for instance in t:
    #loop through the instances and create an attribute for each
    instance.ir_level.ns0_a0 = instance.ir_level.n_0_a_0/instance.ir_level.s_0_a_0
    instance.ir_level.ns1_a0 = instance.ir_level.n_1_a_0/instance.ir_level.s_1_a_0
    instance.ir_level.ns2_a0 = instance.ir_level.n_2_a_0/instance.ir_level.s_2_a_0
    
    #instance.ir_level.ns0_a1 = instance.ir_level.n_0_a_1/instance.ir_level.s_0_a_1
    #instance.ir_level.ns1_a1 = instance.ir_level.n_1_a_1/instance.ir_level.s_1_a_1
    #instance.ir_level.ns2_a1 = instance.ir_level.n_2_a_1/instance.ir_level.s_2_a_1
    
    #instance.ir_level.kn0_a1 = instance.ir_level.k_0_a_1/instance.ir_level.n_0_a_1
    #instance.ir_level.kn1_a1 = instance.ir_level.k_1_a_1/instance.ir_level.n_1_a_1
    #instance.ir_level.kn2_a1 = instance.ir_level.k_2_a_1/instance.ir_level.n_2_a_1
    
    
    #instance.ir_level.y_0_a1 = instance.ir_level.c_0_a_1+instance.ir_level.i_a_1
    
    

In [ ]:
labels = ['No adjustment cost', 'Adjustment cost on labor', 'Adjustment cost on capital']

all_attributes =[   #['y_a_0', 'y_a_1', 'y_a_2'],
                    #['y_0_a_0', 'y_0_a_1', 'y_0_a_2'],
                    #['r_a_0', 'r_a_1', 'r_a_2'],
                    #['tot_a_0', 'tot_a_1', 'tot_a_2'],
                    #['sc_a_0', 'sc_a_1', 'sc_a_2'],
                    #['win_a_0', 'win_a_1', 'win_a_2'],
                    #['k_0_a_0', 'k_1_a_0', 'k_2_a_0'],
                    #['n_0_a_0', 'n_1_a_0', 'n_2_a_0'],
                    #['n_0_a_1', 'n_1_a_1', 'n_2_a_1'],
                    #['s_0_a_0', 's_1_a_0', 's_2_a_0'],
                    #['k_a_0', 'i_a_0'],
                    #['k_0_a_1', 'k_1_a_1', 'k_2_a_1'],
                    #['c_0_a_1', 'c_1_a_1', 'c_2_a_1'],
                    #['k_a_1', 'i_a_1'],
                    #['k_0_a_2', 'k_1_a_2', 'k_2_a_2'],
                    #['c_0_a_2', 'c_1_a_2', 'c_2_a_2'],
                    #['k_a_2', 'i_a_2'],
                    #['ns0_a0', 'ns1_a0', 'ns2_a0'],
                    #['ns0_a1', 'ns1_a1', 'ns2_a1'],
                   # ['kn0_a1', 'kn1_a1', 'kn2_a1'],
                    ['tot_a_0','sc_a_0','win_a_0'],
                    ['ex_0_a_0','ex_1_a_0','ex_2_a_0'],
                    ['exb_0_a_0','exb_1_a_0','exb_2_a_0']
                    
                     ]
for attributes in all_attributes:
    plot_dta(t, attributes,periods=10)
    plt.tight_layout()
    plt.show()                  



In [ ]:
y=t[1].ir_level.y_a_0
n_0=t[1].ir_level.n_0_a_0
n_1=t[1].ir_level.n_1_a_0
n_2=t[1].ir_level.n_2_a_0
s_0=t[1].ir_level.s_0_a_0
s_1=t[1].ir_level.s_1_a_0
s_2=t[1].ir_level.s_2_a_0
k_0=t[1].ir_level.k_a_0
k_0=t[1].ir_level.k_0_a_0
k_1=t[1].ir_level.k_1_a_0
k_2=t[1].ir_level.k_2_a_0
y_0=t[1].ir_level.y_0_a_0
c_0=t[1].ir_level.c_0_a_0
c_1=t[1].ir_level.c_1_a_0
c_2=t[1].ir_level.c_2_a_0
i=t[1].ir_level.i_a_0
tot=t[1].ir_level.tot_a_0
sc=t[1].ir_level.sc_a_0
win=t[1].ir_level.win_a_0
ex_0=t[1].ir_level.ex_0_a_0
ex_1=t[1].ir_level.ex_1_a_0
ex_2=t[1].ir_level.ex_2_a_0
exb_0=t[1].ir_level.exb_0_a_0
exb_1=t[1].ir_level.exb_1_a_0
exb_2=t[1].ir_level.exb_2_a_0





In [ ]:
print 

In [ ]:
ylag=np.roll(y,-1)

In [ ]:
print(y[0][0:10])
print(ylag[0][0:10])

In [ ]:
print(y,ylag)

In [ ]:
params

In [ ]:
a=(s_0/n_0)
b=(s_1/n_1)
c=(s_2/n_2)
print((a)[0][1:4])
print((b)[0][1:4])
print((c)[0][1:4])

In [ ]:
a/c

In [ ]:
print(tot[0][0:4])
print(sc[0][0:4])
print(win[0][0:4])

In [ ]:
a=(ex_0*exb_0+ex_1*exb_1+ex_2*exb_2)
a=a
a[0][0:4]

In [ ]:
a=(exb_0+exb_1+exb_2)
a=a
a[0][0:4]

In [ ]:
print(exb_0[0][0:4])
print(exb_1[0][0:4])
print(exb_2[0][0:4])

In [ ]:
a=ex_0[0][0:4]
b=ex_1[0][0:4]
c=ex_2[0][0:4]
d=exb_0[0][0:4]
e=exb_1[0][0:4]
f=exb_2[0][0:4]



print(a)
print(b)
print(c)
print(d)
print(e)
print(f)

In [ ]:
b*c

In [ ]:
e*f

In [ ]:
print(a*d+b*e+c*f)

In [ ]:
print(n_1[0][0:4])
print(n_2[0][0:4])


In [ ]:
out=[]

In [ ]:
import pandas as pd
from IPython.display import display

# Define a function to highlight the 'win' column in light blue
def highlight_win_column(x):
	df = pd.DataFrame('', index=x.index, columns=x.columns)
	if 'win' in df.columns:
		df['win'] = 'background-color: lightblue'
	if 'tot' in df.columns:
		df['tot'] = 'background-color: lightblue'
	if 'sc' in df.columns:
		df['sc'] = 'background-color: lightblue'

	return df

# Define a function to highlight the diagonal in yellow
def highlight_diagonal(x):
	df = pd.DataFrame('', index=x.index, columns=x.columns)
	for i in range(len(df)):
		df.iat[i, i] = 'background-color: yellow'
  
  
  
	return df

# Iterate through the list and process each element
for i in range(len(t)):
	data = t[i].var

	# Convert to DataFrame
	df = pd.DataFrame(data)

	# Set the index and columns names from ss_dict
	df.index = ss_dict.keys()
	df.columns = ss_dict.keys()

	# Apply the highlighting functions to the DataFrame and round to three digits
	styled_df = df.style.apply(highlight_win_column, axis=None).apply(highlight_diagonal, axis=None).format(precision=3)
	
	# Set smaller font size
	styled_df = styled_df.set_table_styles([{
		'selector': 'table',
		'props': [('font-size', '6pt')]
	}])
	
	# Display the styled DataFrame
	display(styled_df)

In [ ]:
Mthing

In [ ]:
sc-tot

In [10]:
dir(t[1])

['Irf_D',
 'Irf_L',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'autocorr',
 'create_irf_attribute_D',
 'create_irf_attribute_L',
 'dict_irfs',
 'dr',
 'dynare_version',
 'endo_simul',
 'exo_det_simul',
 'exo_det_steady_state',
 'exo_simul',
 'exo_steady_state',
 'gui',
 'initial_exo_steady_state',
 'initial_steady_state',
 'initval_series',
 'ir_dif',
 'ir_level',
 'irfs',
 'kurtosis',
 'mean',
 'oo_dict',
 's_list',
 'skewness',
 'steady_state',
 'time',
 'var',
 'var_list',
 'variance_decomposition']

In [11]:
t[1].steady_state

array([[-1.70401427e+00],
       [-9.84498898e-01],
       [-8.10980785e-01],
       [-6.22653623e-01],
       [-1.34955637e+00],
       [-1.58901381e+00],
       [-1.93234625e-01],
       [ 8.88126024e-01],
       [-5.58292089e-01],
       [-9.71265749e-01],
       [-1.62817099e+00],
       [-1.15198653e+00],
       [-1.10236109e+00],
       [-3.18896680e+00],
       [-1.43757430e+00],
       [ 2.66437807e-01],
       [ 1.02844879e-10],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 4.33925437e-01],
       [ 5.57818111e-01],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00]])